## Dense IR
In this notebook, we show how to train a model, index data, and run search using Neural IR  : in this case ColBERT v2.0 (Santhanam et. al, 2022). TODO: add dependencies

In orded to run (almost) instantaneously, we use trivial data sizes of training data and collection to search.

## Dependencies
If not already done, make sure to install PrimeQA with notebooks extras before getting started.

In [1]:
# If you want CUDA 11 uncomment and run this (for CUDA 10 or CPU you can ignore this line).
#! pip install 'torch~=1.11.0' --extra-index-url https://download.pytorch.org/whl/cu113

# Uncomment to install PrimeQA from source (pypi package pending).
# The path should be the project root (e.g. '.' below).
#! pip install .[notebooks]

## Configuration
First, we need to include the required modules.

In [2]:
import os
import tempfile
 
from primeqa.ir.dense.colbert_top.colbert.utils.utils import create_directory, print_message
from primeqa.ir.dense.colbert_top.colbert.infra import Run, RunConfig
from primeqa.ir.dense.colbert_top.colbert.infra.config import ColBERTConfig
from primeqa.ir.dense.colbert_top.colbert.training.training import train
from primeqa.ir.dense.colbert_top.colbert.indexing.collection_indexer import encode
from primeqa.ir.dense.colbert_top.colbert.searcher import Searcher

No CUDA runtime is found, using CUDA_HOME='/opt/share/cuda-10.1/x86_64'


## Training
We will train a ColBERT model using a TSV file containing [query, positive document, negative document] triples. We use the XOR-TyDi dataset, as described here: https://nlp.cs.washington.edu/xorqa/

The path in `test_files_location` below points to the location of files used by the notebook, by default it points to the files used by CI testing.

In [3]:
test_files_location = '../../../tests/resources/ir_dense'
model_type = 'xlm-roberta-base'
with tempfile.TemporaryDirectory() as working_dir:
    output_dir=os.path.join(working_dir, 'output_dir')
text_triples_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct.tsv")

Here is an example of a training file record:

In [4]:
import pandas as pd
from IPython.display import display, HTML
data = pd.read_csv(text_triples_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))

,0,1,2
0,중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?,"Kangxi Emperor The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.","Chiddy Bang new songs from the duo and in November 2009 debuted the group's first free mixtape entitled ""The Swelly Express"". On 28 April 2011 during the first-ever MTV O Music Awards, Anamege broke the Guinness World Record for Longest Freestyle Rap and Longest Marathon Rapping Record by freestyling for 9 hours, 18 minutes, and 22 seconds, stealing the throne from rapper M-Eighty, who originally broke the record in 2009 by rapping for 9 hours, 15 minutes and 15 seconds. Anamege had also beat Canadian rapper D.O. for Longest Marathon Rapping session, the previous record being for 8 hours and 45 minutes."


In [14]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_training',
                'triples': text_triples_fn,
                'model_type': model_type,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'epochs': 1,
                'rank': 0,
                'nranks': 1,
                'amp' : True
            }

Next we train the model, and save it's location in `the latest_model_fn`variable

In [15]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    latest_model_fn = train(colBERTConfig, text_triples_fn, None, None)

{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "cosine",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta-base",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta-base",
    "

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.6.attention.self.query.bias', 'bert.encoder.layer.9.attention.output.dense.bias', 'bert.encoder.layer.3

[Oct 25, 21:53:28] maxsteps: 3
[Oct 25, 21:53:28] 1 epochs of 5 examples
[Oct 25, 21:53:28] batch size: 1
[Oct 25, 21:53:28] maxsteps set to 3
[Oct 25, 21:53:28] start batch idx: 0
[Oct 25, 21:53:28] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Oct 25, 21:53:28] #> Input: $ 중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?, 		 True, 		 None
[Oct 25, 21:53:28] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[Oct 25, 21:53:28] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[Oct 25, 21:53:28] #> XLMR DocTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[Oct 25, 21:53:28] #> Input: $ "Kangxi

[Oct 25, 21:53:30] #>>>> colbert doc ==
[Oct 25, 21:53:30] #>>>>> D: torch.Size([155, 128]), tensor([[-0.1933, -0.0139, -0.3628,  ..., -0.1452, -0.1189, -0.1785],
        [-0.0756,  0.2043, -0.4927,  ..., -0.1633, -0.2681, -0.0607],
        [-0.0516,  0.2256, -0.5057,  ..., -0.2203, -0.2230, -0.0105],
        ...,
        [-0.1702,  0.0041, -0.3880,  ..., -0.1042, -0.1178, -0.1780],
        [-0.1983, -0.0173, -0.3721,  ..., -0.1311, -0.0983, -0.1707],
        [-0.1884,  0.0173, -0.3752,  ..., -0.1619, -0.1253, -0.1812]],
       grad_fn=<SelectBackward0>)
#>>>    30.06 29.8 		|		 0.259999999999998
[Oct 25, 21:53:40] 0 0.5736338496208191
#>>>    30.38 29.59 		|		 0.7899999999999991
[Oct 25, 21:53:49] 1 0.5734368096590042
#>>>    30.65 29.46 		|		 1.1899999999999977
[Oct 25, 21:53:57] 2 0.5731299413610697
#> Saving a checkpoint to /tmp/tmphjntncqj/output_dir/test_training/2022-10/25/21.44.07/checkpoints/colbert.dnn.batch_3.model ..
#> Saving a checkpoint to /tmp/tmphjntncqj/output_dir/tes

## Indexing
Next, we will index a collection of documents, using model representaion from the previous step. 
The collection is a TSV file, containing each document's ID, title, and text.

In [16]:
collection_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_collection_fornum.tsv")

Here is an example document:

In [17]:
data = pd.read_csv(collection_fn, sep='\t', header=0, nrows=1)
display(HTML(data.to_html()))

,id,text,title
0,1,"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.",Kangxi Emperor


Here are the indexer arguments:

In [18]:
args_dict = {
                'root': os.path.join(output_dir,'test_indexing'),
                'experiment': 'test_indexing',
                'checkpoint': latest_model_fn,
                'collection': collection_fn,
                'index_root': os.path.join(output_dir, 'test_indexing', 'indexes'),
                'index_name': 'index_name',
                'doc_maxlen': 180,
                'num_partitions_max': 2,
                'kmeans_niters': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp' : True
            }

Here we run the indexer:

In [19]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    create_directory(colBERTConfig.index_path_)
    encode(colBERTConfig, collection_fn, None, None)




[Oct 25, 21:54:42] #> Creating directory /tmp/tmphjntncqj/output_dir/test_indexing/indexes/index_name 


{
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "index_location": null,
    "nbits": 1,
    "kmeans_niters": 1,
    "num_partitions_max": 2,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 1,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 10,
    "input_arguments": {},
    "model_type": "bert-base-uncased",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "output_dir": null,
    "topK": 100,
    "student_teacher_tempera

/u/franzm/packages/minconda3/envs/primeqa_pyarrow/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/primeqa_pyarrow/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Oct 25, 21:54:54] #>>>> colbert doc ==
[Oct 25, 21:54:54] #>>>>> input_ids: torch.Size([180]), tensor([    0,  9749, 44759,     6, 58745,  7986,     2,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,    

0it [00:00, ?it/s]

[Oct 25, 21:54:57] [0] 		 #> Encoding 7 passages..
[Oct 25, 21:54:59] [0] 		 #> Saving chunk 0: 	 7 passages and 1,220 embeddings. From #0 onward.


1it [00:01,  1.97s/it]

[Oct 25, 21:54:59] offset: 0
[Oct 25, 21:54:59] chunk codes size(0): 1220
[Oct 25, 21:54:59] codes size(0): 1220
[Oct 25, 21:54:59] codes size(): torch.Size([1220])
[Oct 25, 21:54:59] >>>>partition.size(0): 2
[Oct 25, 21:54:59] >>>>num_partition: 2
[Oct 25, 21:54:59] #> Optimizing IVF to store map from centroids to list of pids..
[Oct 25, 21:54:59] #> Building the emb2pid mapping..
[Oct 25, 21:54:59] len(emb2pid) = 1220



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 5373.87it/s]

[Oct 25, 21:54:59] #> Saved optimized IVF to /tmp/tmphjntncqj/output_dir/test_indexing/indexes/index_name/ivf.pid.pt
[Oct 25, 21:54:59] [0] 		 #> Saving the indexing metadata to /tmp/tmphjntncqj/output_dir/test_indexing/indexes/index_name/metadata.json ..


The resulting index files are in `output_dir/test_indexing/indexes/index_name`

## Search
Next, we use the trained model and the index to search the collection, using queries from a TSV query file.

In [20]:
queries_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_queries_fornum.tsv")

Here are the search arguments:

In [36]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_indexing' ,
                'checkpoint': latest_model_fn,
                'model_type': model_type,
                'index_location': os.path.join(output_dir, 'test_indexing', 'indexes', 'index_name'),
                'queries': queries_fn,
                'bsize': 1,
                'topK': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True,
            }

Here we initalize and run the searcher:

In [37]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    searcher = Searcher(args_dict['index_location'], checkpoint=args_dict['checkpoint'], config=colBERTConfig)
    rankings = searcher.search_all(args_dict['queries'], args_dict['topK'])

[Oct 25, 22:12:07] #> base_config.py from_path /tmp/tmphjntncqj/output_dir/test_indexing/indexes/index_name/metadata.json
[Oct 25, 22:12:07] #> base_config.py from_path args loaded! 
[Oct 25, 22:12:07] #> base_config.py from_path args replaced ! 
[Oct 25, 22:12:07] #> base_config.py load_from_checkpoint /tmp/tmphjntncqj/output_dir/test_training/2022-10/25/21.44.07/checkpoints/colbert
[Oct 25, 22:12:07] #> base_config.py load_from_checkpoint /tmp/tmphjntncqj/output_dir/test_training/2022-10/25/21.44.07/checkpoints/colbert/artifact.metadata
[Oct 25, 22:12:07] #> base_config.py from_path /tmp/tmphjntncqj/output_dir/test_training/2022-10/25/21.44.07/checkpoints/colbert/artifact.metadata
[Oct 25, 22:12:07] #> base_config.py from_path args loaded! 
[Oct 25, 22:12:07] #>>>>> at ColBERT name (model type) : /tmp/tmphjntncqj/output_dir/test_training/2022-10/25/21.44.07/checkpoints/colbert
[Oct 25, 22:12:07] #>>>>> at BaseColBERT name (model type) : /tmp/tmphjntncqj/output_dir/test_training/2022-

/u/franzm/packages/minconda3/envs/primeqa_pyarrow/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Oct 25, 22:13:57] #>>>> before linear query ==
[Oct 25, 22:13:57] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1822,  0.1034,  0.0644,  ..., -0.1299,  0.0443, -0.0150],
        [ 0.0746,  0.0515,  0.0609,  ..., -0.0692, -0.1220, -0.0613],
        [ 0.1168,  0.0506,  0.0132,  ...,  0.0023, -0.0073,  0.2034],
        ...,
        [ 0.1733,  0.0921, -0.0351,  ..., -0.2894, -0.0817,  0.0497],
        [ 0.1733,  0.0921, -0.0351,  ..., -0.2894, -0.0817,  0.0497],
        [ 0.1733,  0.0921, -0.0351,  ..., -0.2894, -0.0817,  0.0497]])
[Oct 25, 22:13:57] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2620e-02, -3.6884e-03, -1.0780e-02,  ..., -1.3896e-02,
          1.2237e-02, -8.2247e-03],
        [-3.5070e-03,  3.9557e-03, -2.4896e-02,  ..., -2.6217e-02,
          4.0822e-03, -8.0775e-03],
        [-1.1149e-02, -1.4444e-05,  1.5010e-02,  ..., -1.1314e-02,
         -9.3701e-03, -2.2194e-02],
        ...,
        [-1.0288e-02,  3.2584e-02,  3.3388e-03,  ...,  3.1875e-02,
         

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 37.52it/s]


Here is the search result for our query, containing [query_id, document_id, rank, score]:

In [38]:
rankings.flat_ranking[0]

(-7239279093922981232, 2, 1, 30.691726684570312)

Here is the text of the query record, contataing the ID and text of the query:

In [39]:
with open(queries_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][0]) == line.split()[0]:
            print(line)

-7239279093922981232	중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?



English translation: `Who maintained the throne for the longest time in China?`

Here is the top retrieved document:

In [40]:
with open(collection_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][1]) == line.split()[0]:
            print(line)

2	Yao. The Bamboo Annals says that when Emperor Zhuanxu died, a descendent of Shennong named ShuQe raised a disturbance, but was destroyed by the prince of Sin, who was Ku (GaoXin), a descendant of HuangDi, who then ascended to the throne. In the 45th year, Ku designated the prince of Tang (唐) (his son Yao) as his successor, however upon his death in the 63rd year, his elder son Zhi then took the throne instead, ruling nine years before being deposed and replaced by Yao. Emperor Zhi Di Zhì () or simply Zhì, was a mythological emperor of ancient China.	Emperor Zhi

